# **Laboratory 1 - Intuitive PID**
In this laboratory session, you will get your hands on PID controller design with the cart-pole problem.
The objective is to tune intuitively a PID controller.

## **Instructions**

**Deliverable.**
You should answer the questions of this lab in a short `.pdf` report.

**Deadline.**
The submission deadline is on the **7th of March 2025 at 11:59 PM**.

**Submission.**
Submit your report on [Gradescope](https://www.gradescope.com/) by logging in with your account `@student.uliege.be`. Don't forget to assign the corresponding pages to the different questions when submitting.
If the course does not appear in your dashboard on Gradescope, contact us on [Ed Discussion](https://edstem.org/us/dashboard) quickly. If you are not familiar with submitting your work to Gradescope, you will find all the necessary information in the [online help](https://help.gradescope.com/article/ccbpppziu9-student-submit-work).

**Collaboration policy.**
You can discuss the assignment with other students, but *you must write your own solutions, and write and run your own code*. Copying someone else's solution, or just making trivial changes to avoid copying verbatim, is not acceptable.

**You may just `Run All` and then you can start.**

In [1]:
# @title
from ipywidgets import *
from IPython.display import display
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
# import logging
plt.close("all")
COLAB=False
try :
    %matplotlib widget
except:
    mpl.use('nbagg')
    COLAB=True

mpl.rcParams['lines.linewidth'] = 2.5
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.labelpad'] = 10
plt.rcParams['axes.titlepad'] = 15
plt.rcParams['axes.titlesize'] = 'large'
plt.rcParams['axes.labelsize'] = 'large'
plt.rcParams['axes.labelweight'] = 'bold'

try :
    import control
except:
    print("/!\ python-control package is not installed" )
    !pip install control
    import control

/!\ python-control package is not installed
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.6/549.6 kB 22.4 MB/s eta 0:00:00


In [2]:
# @title
def cartpole_update(t, x, u, params):
    g = 9.81  # gravity
    mc = 1.0  # mass of the cart
    mp = 0.1  # mass of the pole
    l = 0.5   # length of the pole

    # states
    _, x_dot, theta, theta_dot = x
    F = u[0] + u[1]

    # equations of motion
    sin_theta = np.sin(theta)
    cos_theta = np.cos(theta)
    divisor = mc + mp*sin_theta**2

    x_ddot = (F + mp*sin_theta*(l*theta_dot**2 + g*cos_theta))/(divisor)
    theta_ddot = (-F*cos_theta -mp*l*theta**2 * sin_theta*cos_theta - (mc+mp)*g*sin_theta)/(l*divisor)

    return [x_dot, x_ddot, theta_dot, theta_ddot]

def cartpole_output(t, x, u, params):
    return x[2] + u[-1]

SYSTEM = control.nlsys(cartpole_update, cartpole_output, inputs=('u', 'v', 'w'), outputs=(r'$\theta$'),
                         states=('x', 'x_dot', r'$\theta$', 'theta_dot'), name='cartpole')

class OutPlot:
    def __init__(self, **kwargs):
        self.output = Output()
        self._init_plot(**kwargs)

    def __call__(self, *args, **kwds):
        self.output.clear_output(wait=True)
        with self.output:
            self._plot(*args, **kwds)

    def _plot(self, *args, **kwds):
        pass

    def _init_plot(self, *args, **kwds):
        pass

class ResponsePlot(OutPlot):
    def _init_plot(self, fig_name="all", title=None, tau=5e-2, seed=42):

        np.random.seed(seed)
        self.output.layout.height = '100%'
        self.output.layout.width = '50%'
        plt.close(fig_name)
        self.figure = plt.figure(fig_name, figsize=(12,5),
                                 dpi=100, clear=True)
        self.figure.canvas.header_visible = False
        self.ax = self.figure.add_subplot(1,1,1)
        self.ax.set_xlabel("Time [s]")
        self.ax.set_ylabel("Amplitude [rad]")
        self.ax.set_title(title)

        self.final_T = 15
        self.ax.hlines(np.pi, 0, self.final_T, color='r', linestyle='--', zorder=10)
        self.lines, = self.ax.plot([], [], color='blue')
        self.ax.legend(['Reference', '$\Theta$'])
        self.lowpass = control.zpk([], [-1/tau]*3, (-1/tau)**3, dt=0)
        self.highpass = 1 - self.lowpass

        self.time = np.linspace(0, self.final_T, 5000)
        white_noise = control.white_noise(self.time, np.array([[1e-4, 0], [0, 0.1]]))
        self.disturbance = control.forced_response(self.lowpass, T=self.time, U=white_noise[1])
        self.noise = control.forced_response(self.highpass, T=self.time, U=white_noise[0])

        self.parameters = dict(kp=1, ki=0, kd=0, noise=False, disturbance=False, deviation=np.pi/12)

    def _on_change(self, **kwargs):
        self.parameters.update(kwargs)

    def _on_click(self, button):
        self(**self.parameters)

    def _plot(self, kp, ki, kd,
              noise=False, disturbance=False,
              deviation=2*np.pi/12):
        t = self.time
        pi_controller = control.tf([kp], [1]) + control.tf([ki], [1, 0]) + control.tf([kd*1, 0], [1e-3, 1])
        pi_controller.update_names(name='pi_controller')
        open_loop = control.interconnect([SYSTEM, pi_controller],
                                         connections=[['cartpole.u', 'pi_controller.y'],],
                                         inplist=['pi_controller.u', 'cartpole.v', 'cartpole.w'],
                                         outlist=[r'cartpole.$\theta$'],
                                         name='open_loop')

        sum_junc = control.summing_junction(2, 1, name='sum_junc')
        cartpole_pi = control.interconnect([open_loop, sum_junc],
                                           connections=[['sum_junc.u[1]', '-open_loop.y[0]'],
                                                        ['open_loop.u[0]','sum_junc.y[0]']],
                                           inplist=['sum_junc.u[0]', 'open_loop.u[1]', 'open_loop.u[2]'],
                                            outlist=['open_loop.y[0]'],
                                            name='closed_loop')

        inputs = np.zeros((3, len(t)))
        inputs[0]+=np.pi
        if noise:
            inputs[2] = 1.0 * self.noise.y
        if disturbance:
            inputs[1] = 1.0 * self.disturbance.y

        print("Computing ...", end='')
        states = control.input_output_response(cartpole_pi, U=inputs,
                                       X0=[0,0,np.pi - deviation,0], T=t, squeeze=True)

        # Transform the dictionary into printable key-value pairs
        step_info = control.step_info(states.outputs, T=t)
        self.output.clear_output(wait=True)
        # print(*[f"{key}: {value:.05f}" for key, value in step_info.items()], sep='\n')
        self.lines.remove()
        self.lines, = self.ax.plot(t, states.states[2], color='blue')
        min_ax, max_ax = min(min(states.y[0]), np.pi), max(max(states.y[0]), np.pi)
        self.ax.set_ylim([min_ax - 0.1, max_ax + 0.1])
        display(self.figure)

def create_buttons(kp=True, ki=True, kd=False, noise=False):
    param_list=[]
    if kp:
        param_list.append(BoundedFloatText(value=1, min=0, max=100, step=1, description='Kp'))
    if ki:
        param_list.append(BoundedFloatText(value=0, min=0, max=100, step=.1, description='Ki'))
    if kd:
        param_list.append(BoundedFloatText(value=0, min=0, max=100, step=.1, description='Kd'))
    if noise:
        param_list.append(Checkbox(value=False, description='Add measurement noise'))
        param_list.append(Checkbox(value=False, description='Add disturbance'))
    param_list.append(Button(description='Run simulation'))
    box = VBox(param_list, layout=Layout(flex_flow='column', align_items='center'))
    return box, param_list



# **PID controller: Theoretical questions**
Before getting your hands on simulations, answer the following questions:

> ❓**Q1.** Let $P(s)$ denote the plant transfer function. Draw the block diagram of a closed-loop system controlled with a PID controller.
> The controller block should be detailed into its proportional, integral, and derivative actions in frequency domain form.

> ❓**Q2.** Let $r$ and $y$ denote the reference and the output of the controlled system. Based on the block diagram you drew at the previous point, give the expression of the closed-loop transfer function $G_{yr}(s)$.

> ❓**Q3.** Intuitively, what would be the effect of modifying each parameter ($k_P$, $k_I$, and $k_D$) independently?

# **PID controller: Simulations**
In this simulation, you will control the force $f_x$ to apply to the cart. The goal is to maintain the pole at the upright position given initial deviation.

The pole is initially placed at $\pi + \textit{deviation} $ [rad], and the reference angle is set to $\pi$ [rad] to simulate the step response of your system.

The parameters of your PID controller can be changed using the fields below. The simulation outputs a graph depicting the evolution of the reference angle and the actual angle as a function of time.

> ❓**Q4.** In the following simulation, tune the proportional and integral gains $k_P$ and $k_I$.
>
> **a)** Give a set of values for which the actual angle converges towards the reference angle. In the case you can't find any, what's the apparent problem with the PI controller?
>
> **b)** Does the integral action improve the behavior of the control system? Why?


In [3]:
# @title
response_plot = ResponsePlot(fig_name="cartpole", title=r"PI Controller : true $\theta$")
vbox, pi_params = create_buttons(kp=True, ki=True)
button_pi = pi_params[-1]
button_pi.on_click(response_plot._on_click)
kp_pi, ki_pi = pi_params[:2]

box = HBox([response_plot.output, vbox], layout= Layout(width='100%', align_items='center')) if COLAB else vbox
out = interactive_output(response_plot._on_change, {'kp': kp_pi, 'ki': ki_pi})
display(box)

<IPython.core.display.Javascript object>

> ❓**Q5.**  We will now add a derivative action to the system. Tune the parameters of the PID controller. Give a good set of parameters ($k_P$, $k_I$, and $k_D$) for which the actual distance converges quickly towards the reference angle.
> Explain why the derivative action improves the behavior of the controlled system.

In [4]:
# @title
response_plot_pid = ResponsePlot(fig_name="cartpole_pid", title=r"PID Controller : true $\theta$")
vbox_pid, pid_params = create_buttons(kp=True, ki=True, kd=True)
button_pid = pid_params[-1]
button_pid.on_click(response_plot_pid._on_click)
kp_pid, ki_pid, kd_pid = pid_params[:3]

box_pid = HBox([response_plot_pid.output, vbox_pid], layout= Layout(width='100%', align_items='center')) if COLAB else vbox_pid
out_pid = interactive_output(response_plot_pid._on_change, {'kp': kp_pid, 'ki': ki_pid, 'kd': kd_pid})
display(box_pid)

<IPython.core.display.Javascript object>

> ❓**Q6.**  We will keep the same set of parameters you found at the previous point and add some disturbance on the control (the force $f_x$ applied) and measurement noise on the output of the angle sensor.
> Is your design still valid when noise is added? Explain why focusing on your controller components.

In [5]:
# @title
noise_response_plot = ResponsePlot(fig_name="cartpole_noise", title=r"PID Controller with noise and disturbance: true $\theta$")
vbox_noise, noise_params = create_buttons(kp=True, ki=True, kd=True, noise=True)
button_noise = noise_params[-1]
button_noise.on_click(noise_response_plot._on_click)
kp_noise, ki_noise, kd_noise, measure_noise, disturbance = noise_params[:-1]

box_noise = HBox([noise_response_plot.output, vbox_noise], layout= Layout(width='100%', align_items='center')) if COLAB else vbox_noise
out_noise = interactive_output(noise_response_plot._on_change, {'kp': kp_noise, 'ki': ki_noise, 'kd': kd_noise,
                                                     'noise': measure_noise, 'disturbance': disturbance})
display(box_noise)

<IPython.core.display.Javascript object>